# Runtime: Run Your Model
To test converted or compiled models, arachne has the runtime that wraps original runtimes.  
Currently, the arachne runtime supports the onnx, tflite, and tvm model.

## Prepare and compile a model
We compile mobilenet tf.keras model in tvm. Here tvm_target is set to the CPU as default value.

In [1]:
import tensorflow as tf
model = tf.keras.applications.mobilenet.MobileNet()
model.save("mobilenet.h5")

Save model input and output information as yaml format for arachne.

In [2]:
yml = """
inputs:
  - dtype: float32
    name: input
    shape:
    - 1
    - 224
    - 224
    - 3
outputs:
  - dtype: float32
    name: output
    shape:
    - 1
    - 1000
"""
open("mobilenet.yml", "w").write(yml)


166

In [ ]:
!python -m arachne.driver.cli \
    +tools=tvm \
    model_file=./mobilenet.h5 \
    output_path=./mobilenet.tar \
    model_spec_file=./mobilenet.yml

## Run model on local
Run compiled tvm model using `arachne.runtime`.

In [5]:
import numpy as np

from tvm.contrib.download import download

import arachne.runtime
import arachne.runtime.rpc
import arachne.tools.tvm
import cv2

def get_input_data():
    image_url = "https://arachne-public-pkgs.s3.ap-northeast-1.amazonaws.com/data/cat.jpg"
    image_path = "cat.jpg"
    download(image_url, image_path)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img - np.array([123.0, 117.0, 104.0])
    img /= np.array([58.395, 57.12, 57.375])
    input_data = img[np.newaxis, :, :, :].astype(np.float32)  # type: ignore
    return input_data

def imagenet_1000_class(idx):
    url = "https://gist.githubusercontent.com/zhreshold/4d0b62f3d01426887599d4f7ede23ee5/raw/596b27d23537e5a1b5751d2b0481ef172f58b539/imagenet1000_clsid_to_human.txt"
    path = "imagenet_1000.txt"
    download(url, path)
    return open(path).readlines()[idx].strip()

# Init runtime by the tar files that arachne.tools output
runtime_module = arachne.runtime.init(runtime="tvm", package_tar="mobilenet.tar")
# or init runtime by specific model files and environment files
# runtime_module = arachne.runtime.init(runtime="tvm", model_file="mobilenet.tar", env_file="env.yaml")

# Set an input
input_data = get_input_data()
runtime_module.set_input(0, input_data)
# Run an inference
runtime_module.run()
# Get a result
out = runtime_module.get_output(0)
print(imagenet_1000_class(np.argmax(out)))

285: 'Egyptian cat',


# RPC: Run Your Model on Remote devices

`arachne.runtime.rpc` provides remote execution function on a device using RPC (remote procedure call).  
We describe an example of running mobilenet on jetson-xavier-nx.


## Compile a model for edge device
Compile the mobilenet model same as previous step for the edge device. We set `jetson-xavier-nx` to `tvm_target`.

In [6]:
!python -m arachne.driver.cli \
    +tools=tvm \
    +tvm_target=jetson-xavier-nx \
    model_file=./mobilenet.h5 \
    output_path=./mobilenet_edge.tar \
    model_spec_file=./mobilenet.yml

2022-04-22 04:24:12.703818: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-22 04:24:20.632856: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 373 MB memory:  -> device: 0, name: NVIDIA Tesla V100-SXM2-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-04-22 04:24:20.636986: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30554 MB memory:  -> device: 1, name: NVIDIA Tesla V100-SXM2-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0
2022-04-22 04:24:20.639957: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /jo

## Start RPC server

Please refer to the [Device: Setup device](https://arachne.gitlab.fixstars.com/arachne/tutorials/runtime.html) for device environment setup.
Start the rpc server using the created venv and arachne: `./setup.sh <env_dirname> <runtime_name> <port>`.
You can specify either tvm, tflite, onnx to `<runtime_name>`.
The following example shows the TVM runtime server running on JetPack 4.6 on port 5051.

```sh
cd arachne/device
./setup.sh jp46 tvm 5051
```

Or, you can also start server as the following:  

```sh
cd arachne/device
source jp46/.venv/bin/activate
python -m arachne.runtime.rpc.server --port 5051 --runtime tvm
```


## Run model on remote devices

To perform remote execution via RPC, initialize RuntimeClient with `arachne.runtime.rpc.init`.  
RuntimeClient has the same methods as RuntimeModule, and requests data i/o and model execution to the RuntimeServer running on the edge device.  
RuntimeServicer has `arachne.runtime.RuntimeModule` instance internally and provides model execution services.

In [21]:
# modify it to your environment
rpc_host = "192.168.xx.yy"
rpc_port = 5051

In [22]:
runtime_module = arachne.runtime.rpc.init(
    runtime="tvm",
    package_tar="mobilenet_edge.tar",
    rpc_host=rpc_host,
    rpc_port=rpc_port
)

# Set an input
input_data = get_input_data()
runtime_module.set_input(0, input_data)
# Run an inference
runtime_module.run()
# Get a result
out = runtime_module.get_output(0)
print(imagenet_1000_class(np.argmax(out)))

285: 'Egyptian cat',


RuntimeModule can also run `benchmark` on remote devices.

In [23]:
print(runtime_module.benchmark())

{'mean': 9.475076675415039, 'std': 5.557855606079102, 'max': 26.13087272644043, 'min': 7.470649242401123}


RuntimeClient locks the server when initialization to block other clients from connecting to the RuntimeServer, and unlocks the server when the RuntimeClient instance is deleted or when the `finalize` method is called.

**Only one client can be connected to one Server at the same time. Using a client in the loop of a data loader running in multiprocess may cause gRPC communication to fail.**

In [24]:
runtime_module.finalize()